In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2 import continuousTimeMarkovStateModel as ctmsm
from msmrd2.potentials import patchyProteinMarkovSwitch
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS

In [ ]:
# Define two particle list of rigid body particles
numparticles = 2
state = 0
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([1.0,0,0]), np.array([-1.,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]
for i in range(numparticles):      
    ptype = i
    part = msmrd2.particleMS(ptype, state,D, Drot, positionList[i], orientationList[i])
    pyPartlist.append(part)
pyPartlist[0].setMSMoff()

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleMSList(pyPartlist)

In [ ]:
# Define CTMSM
MSMtype = 1
ratematrix = np.array([[-5.0,5.0],[6.0,-6.0]])
seed = 0 # If seed = -1 uses random device as seed
markovModel = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([0.5, 1.0])
Drotlist = np.array([0.5, 1.0])
markovModel.setD(Dlist)
markovModel.setDrot(Drotlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 0 # If seed = -1 uses random device as seed
bodytype = 'rigidbody' # Mix indicates there can be different types of rigidbody particles
integrator = odLangevinMS(markovModel, dt, seed, bodytype) 
integrator.setKbT(1.0)

In [ ]:
# Define boundary (choose either spherical or box)
radius = 5.0
sphereBoundary = msmrd2.sphere(radius,'reflective')
integrator.setBoundary(sphereBoundary)

In [ ]:
# Define Patchy Protein potential
sigma = 1.0
strength = 65
patchesCoordinates1 = [np.array([1.,0.,0.]), 
                       np.array([0.,1.,0.]),
                       np.array([-1.,0.,0.]),
                       np.array([0.,-1.,0.]),
                       np.array([0.,0.,1.]),
                       np.array([0.,0.,-1.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
potentialPatchyProteinMS = patchyProteinMarkovSwitch(sigma, strength, patchesCoordinates1, patchesCoordinates2)
integrator.setPairPotential(potentialPatchyProteinMS)

In [ ]:
# Integrate particle list and print only positions 
timesteps = 50000 #50000000
stride = 100 #1000
datafile  = open('../data/vmd/patchyProteinsMS.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(9) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[2], part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[3], part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[4], part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[5], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v6)) + '\n')
            if part.type == 1:
                v0 = part.position
                if part.state == 0:
                    v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                else:
                    v1 = 1.0*v0
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteinsMS(numparticles = numparticles)
print("Percentage complete: ", 100, " %")